In [1]:
import json
import re
import os
import tqdm

In [2]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

In [3]:
# 파일 위치 지정
keys_file_path = os.path.join('data', 'api_keys.txt')

# 파일에서 API 키를 로드하는 함수
def load_api_keys(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        keys = json.load(file)
    return keys

# API 키 사용
api_keys = load_api_keys(keys_file_path)

os.environ["GOOGLE_CSE_ID"] = api_keys['google_cse_id']
os.environ["GOOGLE_API_KEY"] = api_keys['google_api_key']

In [4]:
## Web search
search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

## Ollama
llm = ChatOllama(model="llama2")

## CSV Parser
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

In [5]:
# 템플릿 생성
template = """
Information: 
{information}
###
Question: 
{query}
###
Output format:
{format_instructions}
###
System:
{system}
"""

prompt = PromptTemplate.from_template(
    template=template,
    partial_variables={"format_instructions":  format_instructions}
    )

chain = prompt | llm | output_parser

In [6]:
# national = "Korean"
# facet = "food" # food, drink, clothing

In [7]:
import re
import json

national_list = ["Turkish", "Egyptian", "Korean", "Thai", "Mexican", "Spanish"]
facet_list = ["food", "drink", "clothing"]

# 각 facet에 대한 일반적인 단어 리스트
general_terms = {
    "food": ["bread", "rice", "soup", "salad"],
    "drink": ["water", "coffee", "tea", "milk"],
    "clothing": ["shirt", "pants", "dress", "shoes"]
}

national_keyword_dict = {}

for facet in facet_list:
    national_keyword_dict[facet] = {}
    
    for national in national_list:
        system_prompt = "Limit speaking to anything beyond what is asked. !!DO NOT SPEAK SURE!!"
        search_result = tool.run(f"{national} traditional {facet}")
        
        # 일반적인 단어 리스트를 문자열로 변환
        general_terms_str = ', '.join(general_terms[facet])
        query = f"Extract proper nouns of traditional {national} {facet} from the provided information. Exclude common {facet} terms such as {general_terms_str}. Provide the names according to the output format and ensure they are specific to {national} culture."

        # 라마로 키워드 추출
        results = set(chain.invoke({"system": system_prompt, "information": search_result, "query": query}))

        # 출력된 내용에서 한번 더 후처리
        proc_set = set()
        for item in results:
            # 특수 문자 제거, 소문자 변환, 앞뒤 공백 제거
            cleaned_item = re.sub(r'[^a-zA-Z0-9\n\s/]', '', item).lower().strip()
            # '\n'이 있는 경우 분할하여 저장
            if '\n' in cleaned_item:
                cleaned_item_list = cleaned_item.split('\n')
                for c in cleaned_item_list:
                    proc_set.add(c.strip())
            else:
                proc_set.add(cleaned_item)
        # 키워드 길이 0~20까지만 유지
        filtered_results = [item for item in proc_set if 0 < len(item) < 20]

        national_keyword_dict[facet][national] = filtered_results

        print(national, ": ", filtered_results)

print(national_keyword_dict)

# augment_dict를 JSON 형태의 문자열로 변환합니다.
national_keyword_dict_str = json.dumps(national_keyword_dict, ensure_ascii=False, indent=4)

# 파일을 쓰기 모드로 엽니다. 'w'를 사용하여 파일이 이미 존재할 경우 내용을 덮어씁니다.
with open('national_keyword.json', 'w', encoding='utf-8') as file:
    # 변환된 JSON 문자열을 파일에 씁니다.
    file.write(national_keyword_dict_str)


Turkish :  ['taratorlu piyaz', 'kabak iei dolmas', 'tarhana orbas', 'manti', 'baba ghanoush', 'kekek']
Egyptian :  ['mulukhiyah', 'falafel', 'mahshi', 'kofta', 'ful medames', 'koshari', 'ful wa taameya', 'baba ganoush', 'bamia', 'fatta']
Korean :  ['tteokbokki', 'bibimbap', 'pajeon', 'japchae', 'hotteok', 'juk', 'gimbap/kimbap']
Thai :  ['yam nua beef salad', 'gaeng massaman', 'khao soi', 'pad kra pao moo', 'pad thai', 'tom kha gai', 'khao pad', 'som tam', 'tom yum goong']
Mexican :  ['mole poblano', 'barbao', 'pozole', 'chiltomate', 'aguachile', 'chilaquiles', 'chiles en nogada', 'tlayudas', 'carnitas', 'tamales']
Spanish :  ['paella valenciana', 'patatas bravas', 'tortilla espaola', 'gazpacho', 'gambas al ajillo', 'pollo al ajillo']
Turkish :  ['ayran', 'algam', 'raki']
Egyptian :  ['asab', 'karkade', 'khoshaf', 'sugarcane juice']
Korean :  ['gamhongno', 'sikhye', 'dongdongju', 'soju', 'makgeoli']
Thai :  ['september 8', 'august 5', 'plus some sweetened', 'condensed milk', 'august 16

In [8]:
## 증강할 캡션 문장 로드
with open('data/sentences.json', 'r') as f:
    sentences = json.load(f)

print(sentences)

FileNotFoundError: [Errno 2] No such file or directory: 'data/sentences.json'

In [ ]:
facet = "food"
augment_dict = {facet: {}}

for sentence in sentences[facet]:
    for national, keywords in national_keyword_dict[facet].items():
        if national not in augment_dict[facet]:
            augment_dict[facet][national] = []
        for keyword in keywords[:3]:      
            augmented_sentence = sentence.format(national_keyword=keyword)
            print(augmented_sentence)
            augment_dict[facet][national].append(augmented_sentence)

# augment_dict를 JSON 형태의 문자열로 변환합니다.
augment_dict_str = json.dumps(augment_dict, ensure_ascii=False, indent=4)

# 파일을 쓰기 모드로 엽니다. 'w'를 사용하여 파일이 이미 존재할 경우 내용을 덮어씁니다.
with open('augmented_sentences.json', 'w', encoding='utf-8') as file:
    # 변환된 JSON 문자열을 파일에 씁니다.
    file.write(augment_dict_str)